In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import functional as TF
import random
import os
from torchvision import transforms
import shutil
from tqdm.notebook import tqdm
from PIL import Image, ImageFile
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
from sklearn.model_selection import train_test_split

class BikeBicycleDataset(Dataset):
    def __init__(self, root_dir, transform=None, train=True, val_split=0.2):
        self.root_dir = root_dir
        self.transform = transform
        self.train = train
        self.images = []
        self.labels = []

        # Load all the images and labels
        for label, folder in enumerate(['bike', 'motorbike']):
            folder_path = os.path.join(self.root_dir, folder)
            for image_name in os.listdir(folder_path):
                self.images.append(os.path.join(folder_path, image_name))
                self.labels.append(label)

        # Use the correct set for this instance
        if self.train:
            self.images, _, self.labels, _ = train_test_split(
                self.images, self.labels, test_size=(len(self.images) - 500) / len(self.images), stratify=self.labels, random_state=42)
        else:
            _, self.images, _, self.labels = train_test_split(
                self.images, self.labels, test_size=val_split, stratify=self.labels, random_state=42)

        self.data = self.images if self.train else self.images
        self.labels = self.labels if self.train else self.labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        try:
            image_path = self.data[idx]
            label = self.labels[idx]
            image = Image.open(image_path)

            if image.mode == 'P':
                image = image.convert('RGBA')

            image = image.convert('RGB')

            if self.transform:
                image = self.transform(image)
        except (IOError, OSError):
            print(f'Error loading image {image_path}, skipping.')
            idx = (idx + 1) % len(self.data)  # Skip to the next image

        return image, label

# Assuming you have already defined transform_train and transform_val

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = BikeBicycleDataset(root_dir='/content/drive/MyDrive/Bike_and_motorbike', transform=transform_train, train=True)
val_dataset = BikeBicycleDataset(root_dir='/content/drive/MyDrive/Bike_and_motorbike', transform=transform_val, train=False)
test_dataset = BikeBicycleDataset(root_dir='/content/drive/MyDrive/Bike_and_motorbike', transform=transform_test, train=False)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


In [5]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

def resnet18(num_classes=2):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)

In [6]:
model = resnet18(num_classes=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Function to save a checkpoint
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

# Function to save the final model
def save_model(model, filename='final_model.pth'):
    torch.save(model.state_dict(), filename)

# Training function
def train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs=25):
    best_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        with tqdm(enumerate(train_loader), total=len(train_loader), unit="batch") as tepoch:
            for i, (inputs, labels) in tepoch:
                tepoch.set_description(f"Epoch {epoch+1}")
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()
                tepoch.set_postfix(loss=running_loss/total_train, accuracy=100. * correct_train / total_train)

        val_loss, val_acc = validate(model, val_loader, criterion)

        is_best = val_acc > best_acc
        best_acc = max(val_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_acc': best_acc,
        }, is_best)

        print(f'Epoch [{epoch+1}/{num_epochs}] Train Loss: {running_loss/total_train:.4f} Acc: {100. * correct_train / total_train:.4f} Val Loss: {val_loss:.4f} Val Acc: {val_acc:.4f}')

    save_model(model, 'final_model.pth')

    # Testing phase
    test_loss, test_acc = validate(model, test_loader, criterion)
    print(f'Test Loss: {test_loss:.4f} Test Acc: {test_acc:.4f}')

# Validation function
def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        with tqdm(loader, total=len(loader), unit="batch") as tepoch:
            for inputs, labels in tepoch:
                tepoch.set_description("Validation/Test")
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                tepoch.set_postfix(loss=running_loss/total, accuracy=100. * correct / total)

    return running_loss / total, correct / total

# Assuming you have already defined your train_loader, val_loader, and test_loader using the BikeBicycleDataset
train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs=25)

  0%|          | 0/16 [00:00<?, ?batch/s]

  0%|          | 0/76 [00:00<?, ?batch/s]

Epoch [1/25] Train Loss: 0.6314 Acc: 64.4000 Val Loss: 0.7746 Val Acc: 0.4992


  0%|          | 0/16 [00:00<?, ?batch/s]

  0%|          | 0/76 [00:00<?, ?batch/s]

...

  0%|          | 0/16 [00:00<?, ?batch/s]

  0%|          | 0/76 [00:00<?, ?batch/s]

Epoch [25/25] Train Loss: 0.0524 Acc: 98.2000 Val Loss: 0.2829 Val Acc: 0.9134


  0%|          | 0/76 [00:00<?, ?batch/s]

Test Loss: 0.2829 Test Acc: 0.9134
